<a href="https://colab.research.google.com/github/cagBRT/CUDA/blob/main/Nvidia_GPUs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Select the GPU to use by going to<br>

Runtime>Change Runtime Type><br>
Then select a GPU

This output provides information about the NVIDIA GPU that’s currently allocated to your Colab session.

In [1]:
!nvidia-smi

Fri Apr 12 20:10:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Google Colab’s Pros and Cons<br>
**Pros**<br>
Free Access to GPUs: Google Colab provides free access to GPUs, making it an attractive platform for data scientists and software engineers who want to leverage powerful computing resources without incurring additional costs.<br>

Cloud-Based Platform: Being a cloud-based platform, Google Colab eliminates the need for users to set up and maintain their own computing infrastructure. This allows for easier collaboration and seamless sharing of Jupyter notebooks.<br>

Ease of Use: Colab is known for its user-friendly interface and seamless integration with Jupyter notebooks. Users can quickly set up and run machine learning experiments without the hassle of configuring hardware or software.<br>

Wide Adoption: Due to its free GPU access and ease of use, Google Colab has gained widespread adoption among the machine learning community. This popularity fosters a supportive user community and ensures that users can find resources and help easily.

**Cons**<br>
Variable GPU Specs: The allocated GPU specifications in Google Colab can vary, and users may not always have clarity on the available resources. This variability can impact the performance and efficiency of machine learning experiments.<br>

Limited GPU Memory: The allocated GPU memory may not always be sufficient for complex machine learning models or large datasets. Users may face limitations that hinder their ability to train certain types of models or handle extensive datasets.<br>

Resource Availability: Larger GPUs may not always be available, and requesting a change in GPU type or size might result in a longer wait time before the Colab session is ready. This can be a drawback, especially when time is crucial for the experimentation process.



## Error Handling
**Unavailable GPUs**: Users should be prepared to handle situations where GPUs are not available due to high demand or maintenance. It’s advisable to have contingency plans, such as trying the experiment at a later time or exploring alternative platforms.

**Error in GPU Allocation**: If there is an error in GPU allocation, users should check their Colab runtime settings and ensure that they have selected the correct GPU type. Clear instructions on troubleshooting such errors can help users quickly resolve issues.

**Memory Exhaustion**: In cases where GPU memory is insufficient, users should consider optimizing their machine learning models, utilizing data batching techniques, or exploring alternatives like distributed computing. Clear guidance on managing memory issues can enhance the user experience.

**Long Wait Times**: Users requesting larger GPUs should be aware of potential longer wait times. Adequate communication about expected wait times can help manage user expectations and allow them to plan their work accordingly.



Google Colab has a new paid tier, [Pay As You Go](https://colab.research.google.com/signup), giving anyone the option to purchase additional compute time in Colab with or without a paid subscription. This grants access to Colab’s  powerful NVIDIA GPUs and gives you more control over your machine learning environment.

In [2]:
# To show that if there is cuda tookit installed
!ls /usr/local

bin    cuda	cuda-12.2  games	       include	lib64	   man	 share
colab  cuda-12	etc	   _gcs_config_ops.so  lib	licensing  sbin  src


In [3]:
# To show that if we have the nvcc command
!which nvcc

/usr/local/cuda/bin/nvcc


In [4]:
# To show the property of the nvidia card(On my one, I use the K80)
!nvidia-smi

Fri Apr 12 20:10:07 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0              25W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
# Write a cu file contain the host and kernel code
%%writefile hello.cu

#include <cstdio>

__global__ void hello(void)
{
  printf("GPU: Hello!\n");
}

int main(int argc,char **argv)
{
  printf("CPU: Hello!\n");
  hello<<<1,10>>>();
  cudaDeviceReset();
  return 0;
}

Writing hello.cu


In [6]:
#!nvcc --help

In [7]:
!nvcc --list-gpu-arch

compute_50
compute_52
compute_53
compute_60
compute_61
compute_62
compute_70
compute_72
compute_75
compute_80
compute_86
compute_87
compute_89
compute_90


In [8]:
# Compile the code. The flag is needed if you use the Tesla K80.
!nvcc -arch=sm_50 -gencode=arch=compute_50,code=sm_50 hello.cu -o hello

In [9]:
!./hello

CPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!
GPU: Hello!


In [11]:
%%writefile example1.cu

#include <cstdio>
#include <iostream>

using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


Writing example1.cu


In [13]:
!nvcc -arch=sm_50 -gencode=arch=compute_50,code=sm_50 example1.cu -o example1

In [ ]:
!./example1

In [15]:
%%writefile example2.cu

#include <stdio.h>

// This is a special function that runs on the GPU (device) instead of the CPU (host)
__global__ void kernel() {
  printf("Hello world!\n");
}

int main() {
  // Invoke the kernel function on the GPU with one block of one thread
  kernel<<<1,1>>>();

  // Check for error codes (remember to do this for _every_ CUDA function)
  if(cudaDeviceSynchronize() != cudaSuccess) {
    fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(cudaPeekAtLastError()));
  }
  return 0;
}

Writing example2.cu


In [16]:
!nvcc -arch=sm_50 -gencode=arch=compute_50,code=sm_50 example2.cu -o example2

In [17]:
!./example2

Hello world!


In [18]:
%%writefile example3.cu

#include <stdio.h>

// This kernel runs on the GPU and prints the thread's identifiers
__global__ void kernel() {
  printf("Hello from block %d thread %d\n", blockIdx.x, threadIdx.x);
}

int main() {
  // Launch the kernel on the GPU with four blocks of six threads each
  kernel<<<4,6>>>();

  // Check for CUDA errors
  if(cudaDeviceSynchronize() != cudaSuccess) {
    fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(cudaPeekAtLastError()));
  }
  return 0;
}

Writing example3.cu


In [19]:
!nvcc -arch=sm_50 -gencode=arch=compute_50,code=sm_50 example3.cu -o example3

In [20]:
!./example3

Hello from block 3 thread 0
Hello from block 3 thread 1
Hello from block 3 thread 2
Hello from block 3 thread 3
Hello from block 3 thread 4
Hello from block 3 thread 5
Hello from block 0 thread 0
Hello from block 0 thread 1
Hello from block 0 thread 2
Hello from block 0 thread 3
Hello from block 0 thread 4
Hello from block 0 thread 5
Hello from block 2 thread 0
Hello from block 2 thread 1
Hello from block 2 thread 2
Hello from block 2 thread 3
Hello from block 2 thread 4
Hello from block 2 thread 5
Hello from block 1 thread 0
Hello from block 1 thread 1
Hello from block 1 thread 2
Hello from block 1 thread 3
Hello from block 1 thread 4
Hello from block 1 thread 5


In [21]:
%%writefile example4.cu

#include <stdint.h>
#include <stdio.h>

#define N 32
#define THREADS_PER_BLOCK 32

__global__ void saxpy(float a, float* x, float* y) {
  // Which index of the array should this thread use?
  size_t index = 20;

  // Compute a times x plus y for a specific index
  y[index] = a * x[index] + y[index];
}

int main() {
  // Allocate arrays for X and Y on the CPU. This memory is only usable on the CPU
  float* cpu_x = (float*)malloc(sizeof(float) * N);
  float* cpu_y = (float*)malloc(sizeof(float) * N);

  // Initialize X and Y
  int i;
  for(i=0; i<N; i++) {
    cpu_x[i] = (float)i;
    cpu_y[i] = 0.0;
  }

  // The gpu_x and gpu_y pointers will only be usable on the GPU (which uses separate memory)
  float* gpu_x;
  float* gpu_y;

  // Allocate space for the x array on the GPU
  if(cudaMalloc(&gpu_x, sizeof(float) * N) != cudaSuccess) {
    fprintf(stderr, "Failed to allocate X array on GPU\n");
    exit(2);
  }

  // Allocate space for the y array on the GPU
  if(cudaMalloc(&gpu_y, sizeof(float) * N) != cudaSuccess) {
    fprintf(stderr, "Failed to allocate Y array on GPU\n");
    exit(2);
  }

  // Copy the cpu's x array to the gpu with cudaMemcpy
  if(cudaMemcpy(gpu_x, cpu_x, sizeof(float) * N, cudaMemcpyHostToDevice) != cudaSuccess) {
    fprintf(stderr, "Failed to copy X to the GPU\n");
  }

  // Copy the cpu's y array to the gpu with cudaMemcpy
  if(cudaMemcpy(gpu_y, cpu_y, sizeof(float) * N, cudaMemcpyHostToDevice) != cudaSuccess) {
    fprintf(stderr, "Failed to copy Y to the GPU\n");
  }

  // Calculate the number of blocks to run, rounding up to include all threads
  size_t blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

  // Run the saxpy kernel
  saxpy<<<blocks, THREADS_PER_BLOCK>>>(0.5, gpu_x, gpu_y);

  // Wait for the kernel to finish
  if(cudaDeviceSynchronize() != cudaSuccess) {
    fprintf(stderr, "CUDA Error: %s\n", cudaGetErrorString(cudaPeekAtLastError()));
  }

  // Copy the y array back from the gpu to the cpu
  if(cudaMemcpy(cpu_y, gpu_y, sizeof(float) * N, cudaMemcpyDeviceToHost) != cudaSuccess) {
    fprintf(stderr, "Failed to copy Y from the GPU\n");
  }

  // Print the updated y array
  for(i=0; i<N; i++) {
    printf("%d: %f\n", i, cpu_y[i]);
  }

  cudaFree(gpu_x);
  cudaFree(gpu_y);
  free(cpu_x);
  free(cpu_y);

  return 0;
}


Writing example4.cu


In [22]:
!nvcc -arch=sm_50 -gencode=arch=compute_50,code=sm_50 example4.cu -o example4

In [23]:
!./example4

0: 0.000000
1: 0.000000
2: 0.000000
3: 0.000000
4: 0.000000
5: 0.000000
6: 0.000000
7: 0.000000
8: 0.000000
9: 0.000000
10: 0.000000
11: 0.000000
12: 0.000000
13: 0.000000
14: 0.000000
15: 0.000000
16: 0.000000
17: 0.000000
18: 0.000000
19: 0.000000
20: 10.000000
21: 0.000000
22: 0.000000
23: 0.000000
24: 0.000000
25: 0.000000
26: 0.000000
27: 0.000000
28: 0.000000
29: 0.000000
30: 0.000000
31: 0.000000
